In [1]:
import numpy as np
np.seterr(all='raise')
import sys
sys.path.append('../')

from meshmaker.base import Base, Laziness
from meshmaker.model import Model
from meshmaker.mesh import Mesh
from meshmaker.pmesh import ParamMesh, MetaMesh, WireMesh, MetaScene
from meshmaker.meta import Stairs, Railing
from meshmaker.seam import Seam
from meshmaker.tform import TForm
from meshmaker.loop import Loops
from meshmaker.vec3 import vec3
from meshmaker.quat import quat
from meshmaker.planargraph import planargraph
#from meshmaker.geometry import batch, slide, loop_offset, loop_normal, loop_contains, isnear, near, loopO
from meshmaker.geometry import batch, slide, isnear, near, loop_normal, bbox
from meshmaker.mgl import show, MainShader, EdgeShader, WireShader
from meshmaker.plt import *
from collections import defaultdict
import json
from functools import partial

show = partial(show, programs=[MainShader(), EdgeShader(), WireShader(color=vec3.X())])

In [2]:
class Stairs(MetaMesh):
    
    def steps(self, lb, rb):
        h = lb[1].d(lb[0]) 
        z = lb[1].z - lb[0].z
        t = np.arccos(z / h)
        steps = Mesh()
        steps.uvs = {}
        for (aa, bb), (cc, dd) in slide(list(zip(lb, rb)), 2, 1):
            dX = vec3.Z().crs(bb - aa).nrm() * (h * np.sin(t))
            dZ = vec3.Z(lb[1].z - lb[0].z)
            assert dd.z > aa.z, 'Stair splines must point upward!'
            front = steps.af([aa, bb, bb + vec3.Z(dd.z - bb.z), aa + vec3.Z(cc.z - aa.z)])
            top   = steps.af([aa + vec3.Z(cc.z - aa.z), bb + vec3.Z(dd.z - bb.z), dd, cc])
            lside = steps.af([aa, aa + vec3.Z(cc.z - aa.z), cc])
            rside = steps.af([dd, bb + vec3.Z(dd.z - bb.z), bb])
            seams = steps.perimeter((front, top))
            steps.unwrap_uvs(top, O=(aa + vec3.Z(cc.z - aa.z), vec3.O()), S=vec3.U(2),
                             seams=seams, uvs=steps.uvs)            
        return TForm.from_meshes(steps)
    
    def ramp(self, lb, rb, dz=-0.1):
        dz = vec3.Z(dz)        
        ramp = Mesh()
        ramp.af([lb[ 0] + dz, rb[ 0] + dz, rb[ 0], lb[ 0]])
        ramp.af([rb[-1] + dz, lb[-1] + dz, lb[-1], rb[-1]])
        for ((aa, bb), (cc, dd)) in slide(list(zip(lb, rb)), 2, 1):
            ramp.af([cc, aa, bb, dd])
            ramp.af([aa + dz, cc + dz, dd + dz, bb + dz])
            ramp.af([cc + dz, aa + dz, aa, cc])
            ramp.af([dd + dz, dd, bb, bb + dz])
        ramp.uvs = ramp.vertex_uvs(
            O=(lb[0], vec3.O()), S=vec3.U() * 2,
            seams=ramp.angle_seams())
        return TForm.from_meshes(ramp, texture='generic_13')
    
    def splines(self, a, b, c, d, stepheight=0.1, lmargin=0.001, rmargin=0.001):
        r = a.d(d)
        z = d.z - a.z
        n = int(abs(z) / stepheight)        
        N = loop_normal([a, b, c, d])
        
        ldu = N.crs(b - a).nrm()
        ldv = N.crs(d - c).nrm()
        rdu = ldu.cp()
        rdv = ldv.cp()

        dw = (d - a).crs(vec3.Z()).nrm().scl(lmargin)
        lb = Seam((a + dw), (d + dw), n, ldu, ldv).loop()

        dw = (d - a).crs(vec3.Z()).nrm().scl(rmargin)
        rb = Seam((b - dw), (c - dw), n, rdu, rdv).loop()

        if z < 0:
            lb.reverse()
            rb.reverse()

        return lb, rb
    
    def scene(self):
        vs, fs = self.control.vertices, self.control.faces
        scene = TForm()
        for i, ((a, b), (c, d)) in enumerate(self.control._stairs):
            a, b, c, d = vs[a], vs[b], vs[c], vs[d]
            lb, rb = self.splines(a, b, d, c)
            scene.add(self.ramp(lb, rb))            
            if not (isnear(a.z, c.z) and isnear(b.z, d.z)):
                scene.add(self.steps(lb, rb))
        return scene

    
class Railings(MetaMesh):
    
    def railing(self, a, b, dz=0.4, dw=0.02):
        dz = vec3.Z(dz)
        railing = Mesh.prism([a, b, b + dz, a + dz], dw, 2)
        railing.uvs = railing.vertex_uvs(
            O=(a, vec3.O()), S=vec3.U() * 2,
            seams=railing.angle_seams())
        return TForm.from_meshes(railing, texture='generic_9')

    def scene(self):
        vs, fs = self.control.vertices, self.control.faces
        scene = TForm()
        for i, (a, b) in enumerate(self.control._railings):
            scene.add(self.railing(vs[a], vs[b]))
        return scene

In [3]:
def bendy(railing_width=0.02):
    control = Mesh.lattice()
    av = lambda x, y, z: control.av(vec3(x, y, z))
    control._stairs = []
    control._railings = []
    #control._stairs.append(((av(-1,-1,-1), av( 0,-1,-1)),
    #                        (av( 1, 1, 0), av( 1, 0, 0))))
    control._stairs.append(((av( 1, 1, 0), av( 1, 0, 0)),
                            (av(-1,-1,-1), av( 0,-1,-1))))
    #control._railings.append(((av(-1,-1,-1), av(-1, 0, 0))))
    #control._railings.append(((av( 0, 0, 0), av( 0,-1,-1))))
    return control

control = bendy()
show(MetaScene(TForm(metas=[Stairs(control), Railings(control), WireMesh(control)])))

Building Shader 1:<meshmaker.mgl.MainShader object at 0x7f1173f13d68>
Building Shader 2:<meshmaker.mgl.EdgeShader object at 0x7f1173f13f98>
Building Shader 3:<meshmaker.mgl.WireShader object at 0x7f1173fb27b8>
Loaded texture: generic_13 (../resources/textures/generics/generic_13.png)
Loaded texture: generic_8 (../resources/textures/generics/generic_8.png)


In [12]:
class Prefabs:
    
    def _lattice(self, nx, ny, nz):
        c = Mesh.lattice(nx, ny, nz)
        c._stairs = []
        c._railings = []
        return c

    def _tf(self, c, **kws):
        return TForm(metas=[Stairs(c), Railings(c), WireMesh(c)], **kws)
    
    def _3x3x3_z0(self, **kws):
        c = self._lattice(3, 3, 3)
        v = lambda x, y, z: c._fp(vec3(x, y, z))
        c._stairs.append(((v(-1,-1,-1), v( 1,-1,-1)), (v(-1, 1,-1), v( 1, 1,-1))))
        return self._tf(c, **kws)

    def _3x3x3_switchback(self, **kws):
        return self._tf(switchback(), **kws)
    
pf = Prefabs()
tf = TForm()

tf.add(pf._3x3x3_z0(t=vec3( 0, 0, 0)))
tf.add(pf._3x3x3_switchback(t=vec3( 2, 0, 0)))

show(MetaScene(tf))

Building Shader 1:<meshmaker.mgl.MainShader object at 0x7f1173f13d68>
Building Shader 2:<meshmaker.mgl.EdgeShader object at 0x7f1173f13f98>
Building Shader 3:<meshmaker.mgl.WireShader object at 0x7f1173fb27b8>
Loaded texture: generic_13 (../resources/textures/generics/generic_13.png)
Loaded texture: generic_8 (../resources/textures/generics/generic_8.png)
Loaded texture: generic_9 (../resources/textures/generics/generic_9.png)


In [7]:
control = Mesh.cube_mesh()
control._stairs = [((0, 1), (7, 6))]
#control._stairs = [((6, 7), (1, 0))]
control._railings = []
stairs = Stairs(control)
show(MetaScene(TForm(metas=[stairs, WireMesh(control)])))

Building Shader 1:<meshmaker.mgl.MainShader object at 0x7f2ee47a6630>
Building Shader 2:<meshmaker.mgl.EdgeShader object at 0x7f2ee47a6c88>
Building Shader 3:<meshmaker.mgl.WireShader object at 0x7f2ee47af3c8>
Loaded texture: generic_13 (../resources/textures/generics/generic_13.png)
Loaded texture: generic_8 (../resources/textures/generics/generic_8.png)


In [4]:
control = Mesh.cube_mesh()
control._stairs = [((0, 1), (7, 6))]
#control._stairs = [((6, 7), (1, 0))]
control._railings = []
stairs = Stairs(control)
show(MetaScene(TForm(metas=[stairs, WireMesh(control)])))

Building Shader 1:<meshmaker.mgl.MainShader object at 0x7f0ddc536ef0>
Building Shader 2:<meshmaker.mgl.EdgeShader object at 0x7f0ddc536470>
Building Shader 3:<meshmaker.mgl.WireShader object at 0x7f0ddc52f9b0>
Loaded texture: generic_13 (../resources/textures/generics/generic_13.png)
Loaded texture: generic_8 (../resources/textures/generics/generic_8.png)


In [5]:
def escape(railing_width=0.02):
    control = Mesh.lattice(5, 2, 2)
    
    av = lambda x, y, z: control.av(vec3(x, y, z))

    control._stairs = []
    control._railings = []

    control._stairs.append(((av(  -1, 1,-1), av(  -1,-1,-1)),
                            (av(-0.5, 1,-1), av(-0.5,-1,-1))))
    control._railings.append(((av(  -1,-1 + railing_width * 4,-1), av(-1, 1,-1))))
    control._railings.append(((av(-0.5,-1,-1), av(-1,-1,-1))))

    control._stairs.append(((av(-0.5, 1,-1), av(-0.5,-1,-1)),
                            (av( 0.5, 1, 1), av( 0.5,-1, 1))))
    control._railings.append(((av( 0.5,-1, 1), av(-0.5,-1,-1))))
    control._railings.append(((av(-0.5, 1,-1), av( 0.5, 1, 1))))

    control._stairs.append(((av( 0.5, 1, 1), av( 0.5,-1, 1)),
                            (av(   1, 1, 1), av(   1,-1, 1))))
    control._railings.append(((av( 1, 1, 1), av(   1,-1 + railing_width * 4, 1))))
    control._railings.append(((av( 1,-1, 1), av( 0.5,-1, 1))))
    
    return control

control = escape()
vec3(1, 0.25, 0.5).sclps(control.vertices)
show(MetaScene(TForm(metas=[Stairs(control), Railings(control), WireMesh(control)])))

Building Shader 1:<meshmaker.mgl.MainShader object at 0x7f0ddc536ef0>
Building Shader 2:<meshmaker.mgl.EdgeShader object at 0x7f0ddc536470>
Building Shader 3:<meshmaker.mgl.WireShader object at 0x7f0ddc52f9b0>
Loaded texture: generic_13 (../resources/textures/generics/generic_13.png)
Loaded texture: generic_8 (../resources/textures/generics/generic_8.png)
Loaded texture: generic_9 (../resources/textures/generics/generic_9.png)


In [11]:
def switchback(railing_width=0.02):
    #control = Mesh.cube_mesh()
    #control = control.subdivide()
    control = Mesh.lattice()
    
    #av = lambda x, y, z: control._fp(vec3(x, y, z))
    av = lambda x, y, z: control.av(vec3(x, y, z))
    
    control._stairs = []
    control._railings = []

    # bottom ramp
    control._stairs.append(((av(-1,-1,-1), av( 0,-1,-1)),
                            (av(-1, 0, 0), av( 0, 0, 0))))
    control._railings.append(((av(-1,-1,-1), av(-1, 0, 0))))
    control._railings.append(((av( 0, 0, 0), av( 0,-1,-1))))
    
    # top ramp
    control._stairs.append(((av( 1, 0, 0), av( 0, 0, 0)),
                            (av( 1,-1, 1), av( 0,-1, 1))))
    control._railings.append(((av( 1, 0, 0), av( 1,-1, 1))))
    control._railings.append(((av( 0,-1, 1), av( 0, 0, 0))))
    
    # landing
    control._stairs.append(((av(-1, 1, 0), av(-1, 0, 0)),
                            (av( 0, 1, 0), av( 0, 0, 0))))
    control._stairs.append(((av( 0, 1, 0), av( 0, 0, 0)),
                            (av( 1, 1, 0), av( 1, 0, 0))))
    control._railings.append(((av(-1, 0, 0), av(-1, 1, 0))))
    control._railings.append(((av(-1 + railing_width, 1, 0),
                               av( 1 - railing_width, 1, 0))))
    control._railings.append(((av( 1, 1, 0), av( 1, 0, 0))))
    control._railings.append(((av( railing_width, 0, 0),
                               av(-railing_width, 0, 0))))

    return control

def catwalk():
    control = Mesh.lattice(nx=2, ny=2, nz=2)
    av = lambda x, y, z: control.av(vec3(x, y, z))

    control._stairs = []
    control._railings = []
    control._stairs.append(((av(-1,-1,-1), av( 1,-1,-1)),
                            (av(-1, 1,-1), av( 1, 1,-1))))
    control._railings.append((av(-1,-1,-1), av( 1,-1,-1)))

    return control


def test():

    def deform(box):
        controls = {i: v for i, v in enumerate(box.vertices)}
        
        constraints = {}
        x, o = vec3(-1,-1, 1), vec3.U(-1)
        constraints[control._fp(x)] = x + o
        for x in (vec3(-1,-1,-1), vec3( 1,-1,-1), vec3( 1, 1,-1), vec3(-1, 1,-1)):
            constraints[control._fp(x)] = x
        print(constraints)

        return box.deform(constraints, inplace=True)

    sb = switchback()
    x = TForm(t=vec3.O(), metas=[Stairs(sb), Railings(sb), WireMesh(sb)])

    #control = deform(control)

    cw = catwalk()
    y = TForm(t=vec3.Y(-2),
              metas=[Stairs(cw), Railings(cw), WireMesh(cw)])
    z = TForm(t=(vec3.Y(-2) + vec3.Z(2) + vec3.X(0.5)), s=vec3( 0.5, 1, 1),
              metas=[Stairs(cw), Railings(cw), WireMesh(cw)])

    tf = TForm()
    for o in (x, y, z):
        tf.add(o)
    show(MetaScene(tf))


test()

Building Shader 1:<meshmaker.mgl.MainShader object at 0x7f1173f13d68>
Building Shader 2:<meshmaker.mgl.EdgeShader object at 0x7f1173f13f98>
Building Shader 3:<meshmaker.mgl.WireShader object at 0x7f1173fb27b8>
Loaded texture: generic_13 (../resources/textures/generics/generic_13.png)
Loaded texture: generic_8 (../resources/textures/generics/generic_8.png)
Loaded texture: generic_9 (../resources/textures/generics/generic_9.png)


In [9]:
control = switchback()
show(MetaScene(TForm(metas=[Stairs(control), Railings(control), WireMesh(control)])))

Building Shader 1:<meshmaker.mgl.MainShader object at 0x7f3f4d4fe4a8>
Building Shader 2:<meshmaker.mgl.EdgeShader object at 0x7f3f4d4fea90>
Building Shader 3:<meshmaker.mgl.WireShader object at 0x7f3f4d5071d0>
Loaded texture: generic_13 (../resources/textures/generics/generic_13.png)
Loaded texture: generic_8 (../resources/textures/generics/generic_8.png)
Loaded texture: generic_9 (../resources/textures/generics/generic_9.png)


# MAKE A LATTICE CLASS TO SERVE AS BASE CONTROL MESH WHICH DEFORMS NICELY BECAUSE CONNECTIVITY IS PREDICTABLE 

In [10]:
def rapture():
    control = Mesh()
    control._stairs = []
    control._railings = []
    av = lambda x, y, z: control.av(vec3(x, y, z))
    
    topfloor = control.af([av(-1,-1, 1), av( 0,-1, 1),
                           av( 0, 1, 1), av(-1, 1, 1)])
    botfloor = control.af([av( 0,-1, 0), av( 2,-1, 0),
                           av( 2, 1, 0), av( 0, 1, 0)])

    a, b, c, d = (av( 1,-1, 0), av( 1, 0 - 0.5, 0),
                  av( 0,-1, 1), av( 0, 0 - 0.5, 1))
    control._stairs.append(((a, b), (c, d)))
    control._railings.append((a, c))
    control._railings.append((d, b))
    
    a, b, c, d = (av( 1, 0 + 0.5, 0), av( 1, 1, 0),
                  av( 0, 0 + 0.5, 1), av( 0, 1, 1))
    control._stairs.append(((a, b), (c, d)))
    control._railings.append((a, c))
    control._railings.append((d, b))
    
    metas = [MetaMesh(control), Stairs(control), Railings(control)]
    return MetaScene(TForm(metas=metas))
    
show(rapture())

Building Shader 1:<meshmaker.mgl.MainShader object at 0x7f3f4d4fe4a8>
Building Shader 2:<meshmaker.mgl.EdgeShader object at 0x7f3f4d4fea90>
Building Shader 3:<meshmaker.mgl.WireShader object at 0x7f3f4d5071d0>
Loaded texture: generic_13 (../resources/textures/generics/generic_13.png)
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)
Loaded texture: generic_8 (../resources/textures/generics/generic_8.png)
Loaded texture: generic_9 (../resources/textures/generics/generic_9.png)


In [26]:
a = Mesh.cube_mesh()
vec3.Z(-1).trnps(vec3(2, 2, 1).sclps(a.vertices))

b = Mesh.cube_mesh()

c = Mesh.cube_mesh()
vec3(0.5, 3, 1).sclps(c.vertices)

show(a.difference(b).difference(c))

Building Shader 1:<meshmaker.mgl.MainShader object at 0x7fce4aa274a8>
Building Shader 2:<meshmaker.mgl.EdgeShader object at 0x7fce4aa27a90>
Building Shader 3:<meshmaker.mgl.WireShader object at 0x7fce4aa171d0>
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)


In [27]:
control = Mesh()

rim = vec3.O().ring(4, 4, False)
irim = Loops([rim]).offset(1).loops[0]

floor = control.bridge(rim, irim)
#floor = control.edgeloopsplit(0, 1)

show(control)

Building Shader 1:<meshmaker.mgl.MainShader object at 0x7fce4aa274a8>
Building Shader 2:<meshmaker.mgl.EdgeShader object at 0x7fce4aa27a90>
Building Shader 3:<meshmaker.mgl.WireShader object at 0x7fce4aa171d0>
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)
